In [1]:
import requests
import pyarrow as pa
import pyarrow.fs
import io

In [2]:
#Q1:
!hdfs dfsadmin -fs hdfs://boss:9000 -report

Configured Capacity: 51642105856 (48.10 GB)
Present Capacity: 20401545216 (19.00 GB)
DFS Remaining: 20401496064 (19.00 GB)
DFS Used: 49152 (48 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.24.0.3:9866 (project-4-asian-cheese-1-dn-2.project-4-asian-cheese-1_default)
Hostname: 56bca2a3d1a2
Decommission Status : Normal
Configured Capacity: 25821052928 (24.05 GB)
DFS Used: 24576 (24 KB)
Non DFS Used: 15603503104 (14.53 GB)
DFS Remaining: 10200748032 (9.50 GB)
DFS Used%: 0.00%
DFS Remaining%

In [3]:

!wget -nc https://pages.cs.wisc.edu/~harter/cs544/data/hdma-wi-2021.csv
!hdfs dfs -D dfs.block.size="1M" -D dfs.replication=1 -cp hdma-wi-2021.csv hdfs://boss:9000/single.csv
!hdfs dfs -D dfs.block.size="1M" -D dfs.replication=2 -cp hdma-wi-2021.csv hdfs://boss:9000/double.csv

File ‘hdma-wi-2021.csv’ already there; not retrieving.



In [4]:
#Q2:
!hdfs dfs -du -h hdfs://boss:9000/single.csv
!hdfs dfs -du -h hdfs://boss:9000/double.csv


166.8 M  166.8 M  hdfs://boss:9000/single.csv
166.8 M  333.7 M  hdfs://boss:9000/double.csv


In [5]:
#Q3:
u="http://boss:9870/webhdfs/v1/single.csv?op=GETFILESTATUS"
r = requests.get(u)

r.json()

{'FileStatus': {'accessTime': 1710574034718,
  'blockSize': 1048576,
  'childrenNum': 0,
  'fileId': 16386,
  'group': 'supergroup',
  'length': 174944099,
  'modificationTime': 1710574040908,
  'owner': 'root',
  'pathSuffix': '',
  'permission': '644',
  'replication': 1,
  'storagePolicy': 0,
  'type': 'FILE'}}

In [6]:
#Q4:
web_u = "http://boss:9870/webhdfs/v1/single.csv?op=OPEN&noredirect=true&offset=0"

response = requests.get(web_u)
data = response.json()

loc_u = data.get('Location', '')

loc_u

'http://56bca2a3d1a2:9864/webhdfs/v1/single.csv?op=OPEN&namenoderpcaddress=boss:9000&offset=0'

In [7]:
#Q5:
web_u_5 = "http://boss:9870/webhdfs/v1/single.csv?op=GET_BLOCK_LOCATIONS"

response = requests.get(web_u_5 )
data = response.json()
my_map= {}

if 'LocatedBlocks' in data and 'locatedBlocks' in data['LocatedBlocks']:
    
    for block in data['LocatedBlocks']['locatedBlocks']:
        
        for loc in block['locations']: #get id for hostname/IP
            dn_id = loc['hostName']
            
            if dn_id not in my_map:
                my_map[dn_id] = 0
            my_map[dn_id] +=1

my_map


{'56bca2a3d1a2': 84, '0b534b8c4501': 83}

In [8]:
#Q6:
hdfs = pa.fs.HadoopFileSystem("boss", 9000)
file = hdfs.open_input_file("hdfs://boss:9000/single.csv")
file.read_at(10, 0)

2024-03-16 07:27:39,042 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


b'activity_y'

In [9]:
#Q7:
counter = 0
file = hdfs.open_input_file("hdfs://boss:9000/single.csv")
reader = io.TextIOWrapper(io.BufferedReader(file))

for i, j in enumerate(reader):
    
    if "Single Family" in j:
        counter+=1

counter


444874